In [453]:
from bs4 import BeautifulSoup as soup
import requests
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [13]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

AttributeError: 'str' object has no attribute 'json'

In [251]:
wiki=requests.get(url).text
wiki_clean=soup(wiki, 'lxml')
wiki_clean

toronto=wiki_clean.find('table')

table_contents=[]

In [252]:
for row in toronto.findAll('td'):
    cell={}
    if row.span.text=='Not Assigned':
        pass
    else:
        cell['PostalCode']=row.p.text[:3]
        cell['Borough']=(row.span.text).split('(')[0]
        try:
            cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        except:
            pass
        table_contents.append(cell)


In [254]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [256]:
df.drop(df.index[1])
to_delete=[]

for i in range(len(df['Borough'])):
    if 'Not assigned' in df[['Borough']].iloc[i].values[0]:
        to_delete.append(i)

df_clean=df.drop(df.index[to_delete])


In [244]:
for i in range(len(df_clean['Neighborhood'])):
    try:
        if math.isnan(df_clean[['Neighborhood']].iloc[i].values[0] ):
            df_clean['Neighborhood'].iloc[i]=df_clean['Borough'].iloc[i]

    except:
        pass

In [245]:
# alternative： 
#df[['Borough']].iloc[i].values[0]=='Not assigned' or df[['Borough']].iloc[i].values[0]=='[Not assigned]':

In [262]:
df_clean.head(100)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills North
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [260]:
df_toronto=df_clean

In [261]:
df_toronto

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills North
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [268]:
tolatlng=pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

In [269]:
tolatlng


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [293]:
mergedto=pd.merge(left=df_toronto, right=tolatlng, left_on='PostalCode', right_on='Postal Code')
mergedto.drop(columns='Postal Code', inplace=True, axis=1)
torontodf=mergedto

In [281]:
mergedto.head()
# we have finished processing the dataframe, now we begin on foursqures.


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [291]:
address='Toronto, ON'

geolocator=Nominatim(user_agent='Toronto_locator') #user_agent is user defined app name

location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The latitude and longitude of Toronto are respectively:{} and {}'.format(latitude, longitude))

The latitude and longitude of Toronto are respectively:43.6534817 and -79.3839347


## Creating a map of Toronto

In [302]:
toronto_map=folium.Map(location=[latitude, longitude],zoom_start=10)
for lat, lng, borough, neighborhood in zip(torontodf['Latitude'], torontodf['Longitude'], torontodf['Borough'], torontodf['Neighborhood']):
    label='{},{}'.format(neighborhood, borough)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=4,
        color='white',
        popup=label,
        fill=True,
        fill_color='#FF0000',
        fill_opacity=0.5,
        parse_html=False).add_to(toronto_map) 
    
toronto_map

In [296]:
torontodf.loc[0, 'N']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## FourSquares:

In [303]:
#USER Credentials:
CLIENT_ID='PMX3QDWCECWBGL1M1LUGOSE5YKFP1VIAD35TOMRRNGSUWQ5I'
CLIENT_SECRET='S1WXZRWXXLW5IOYL5KIKDMJN4JLCFQ51MH5XQ3CALVEDW42T'
VERSION='20180605'
LIMIT=100

ACCESS_TOKEN='N2LYR4HES2AJZSNMQBONHR5TOCNRASOGXJN3JK4W1ZLULBLK'
print('Your Creditials:\n'  'CLIENT_ID:'+CLIENT_ID,'\n' 'CLIENT_SECRET:'+CLIENT_SECRET)

Your Creditials:
CLIENT_ID:PMX3QDWCECWBGL1M1LUGOSE5YKFP1VIAD35TOMRRNGSUWQ5I 
CLIENT_SECRET:S1WXZRWXXLW5IOYL5KIKDMJN4JLCFQ51MH5XQ3CALVEDW42T


In [304]:
radius=500


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

NameError: name 'neighborhood_latitude' is not defined

In [312]:
#since there are too many venus in Toronto, we will do as adviced and only include Boroughs which includes the word 'Toronto。'

In [313]:
Boroughs=[]
for i in range(len(torontodf)):
    if 'Toronto' in torontodf['Borough'].iloc[i]:
        Boroughs.append(i)

In [329]:
new_torontodf=pd.DataFrame(torontodf.iloc[Boroughs])

In [334]:
new_torontodf.reset_index(inplace=True)

In [336]:
new_torontodf.drop(columns='index', inplace=True)

In [338]:
new_torontodf

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
9,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106


### New_torontodf will be the dataframe we explore because it has been filtered to keep Boroughs that contain the word Toronto only

### First we define the function to get nearby values.

In [351]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print('Neighborhood:', name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results=requests.get(url).json()['response']['groups'][0]['items']
        
        for venue in results:
            venues_list.append([(
            name, lat, lng,
            venue['venue']['name'],
            venue['venue']['location']['lat'],
            venue['venue']['location']['lng'],
            venue['venue']['categories'][0]['name'])])
        
        nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        
        nearby_venues.columns=['Neighborhood','Neighborhood Latitude','Neighborhood Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category']
    return(nearby_venues)

In [352]:
Toronto_venues=getNearbyVenues(new_torontodf['Neighborhood'],new_torontodf['Latitude'], new_torontodf['Longitude'])

Neighborhood: Regent Park, Harbourfront
Neighborhood: Garden District, Ryerson
Neighborhood: St. James Town
Neighborhood: The Beaches
Neighborhood: Berczy Park
Neighborhood: Central Bay Street
Neighborhood: Christie
Neighborhood: Richmond, Adelaide, King
Neighborhood: Dufferin, Dovercourt Village
Neighborhood: The Danforth  East
Neighborhood: Harbourfront East, Union Station, Toronto Islands
Neighborhood: Little Portugal, Trinity
Neighborhood: The Danforth West, Riverdale
Neighborhood: Toronto Dominion Centre, Design Exchange
Neighborhood: Brockton, Parkdale Village, Exhibition Place
Neighborhood: India Bazaar, The Beaches West
Neighborhood: Commerce Court, Victoria Hotel
Neighborhood: Studio District
Neighborhood: Lawrence Park
Neighborhood: Roselawn
Neighborhood: Davisville North
Neighborhood: Forest Hill North & West
Neighborhood: High Park, The Junction South
Neighborhood: North Toronto West
Neighborhood: The Annex, North Midtown, Yorkville
Neighborhood: Parkdale, Roncesvalles
Neig

In [357]:
Toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


## We got the dataframe of nearby venues as shown above.


In [360]:
Toronto_venues.groupby('Neighborhood').count().head()
#This shows us how many venues there are in each Neighborhood:


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,60,60,60,60,60,60
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,61,61,61,61,61,61
Christie,16,16,16,16,16,16


### Get dummies for the categories of these venues in each neighborhood.

In [409]:
Toronto_onehot=pd.get_dummies(Toronto_venues[['Venue Category']], prefix='', prefix_sep='')
Toronto_onehot['Neighborhood']=Toronto_venues['Neighborhood']
columns=[]
for column in Toronto_onehot.columns:
    if column !='Neighborhood':
        columns.append(column)
onehot_columns=['Neighborhood']+columns

Toronto_onehot=Toronto_onehot[onehot_columns]
Toronto_onehot.head()

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [383]:
Toronto_onehot.shape

(1575, 237)

In [410]:
Toronto_grouped=Toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [412]:
Toronto_grouped.head()

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.016667,0.0,0.0,0.000000,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,Central Bay Street,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.016393,0.0,0.0,0.016393,0.0,0.016393
4,Christie,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000


In [430]:
def return_most_common_venues(row, num_top_venues):
    row_categories=row.iloc[1:] #0 is excluded because 0 is Neighborhood
    
    row_categories_sorted=row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
    
    

In [436]:
num_top_venues=5
indicators=['st','nd','rd']

columns1=['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns1.append('{}{} Most Common type of Venue'.format(ind+1, indicator[ind]))
    except:
        columns1.append('{}th Most common type of Venue'.format(ind+1))
    
Toronto_venues_sorted=pd.DataFrame(columns=columns1)
Toronto_venues_sorted['Neighborhood']=Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    Toronto_venues_sorted.iloc[ind, 1:]=return_most_common_venues(Toronto_grouped.iloc[ind,:], num_top_venues)

#Toronto_venues_sorted.head()

In [437]:
np.arange(5)

array([0, 1, 2, 3, 4])

In [439]:
Toronto_venues_sorted.head()

,Neighborhood,1th Most common type of Venue,2th Most common type of Venue,3th Most common type of Venue,4th Most common type of Venue,5th Most common type of Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Beer Bar
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Convenience Store,Intersection
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Boutique
3,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place
4,Christie,Grocery Store,Café,Park,Baby Store,Coffee Shop


# Clustering:

In [482]:
kclusters=5

Toronto_grouped_clustering=Toronto_grouped.drop('Neighborhood', 1)

kmeans=KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

kmeans.labels_[0:10]



Toronto_venues_sorted.insert(0, 'Cluster labels', kmeans.labels_)
Toronto_merged=pd.merge(left=new_torontodf, right=Toronto_venues_sorted, left_on='Neighborhood', right_on='Neighborhood')



In [481]:
#Toronto_venues_sorted.drop('Cluster labels', inplace=True, axis=1)

In [483]:
Toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster labels,1th Most common type of Venue,2th Most common type of Venue,3th Most common type of Venue,4th Most common type of Venue,5th Most common type of Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Pub,Bakery,Park,Café
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Cosmetics Shop,Cocktail Bar,Hotel
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Asian Restaurant,Pub,Health Food Store,Trail,Discount Store
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Beer Bar


In [484]:
clusters_map = folium.Map(location=[latitude, longitude], zoom_start=10)

x=np.arange(kclusters)
y=range(kclusters)
colors_array=cm.rainbow(np.linspace(0,1,len((y))))

rainbow=[colors.rgb2hex(i) for i in colors_array]

marker_colors=[]

for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'],\
                                 Toronto_merged['Cluster labels']):
    label=folium.Popup(str(poi)+'Cluster'+str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius=4,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster],
        fill_opacity=0.5).add_to(clusters_map)
clusters_map

## Cluster 1

In [488]:
Toronto_merged.loc[Toronto_merged['Cluster labels']==0, Toronto_merged.columns[[1]+list (range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster labels,1th Most common type of Venue,2th Most common type of Venue,3th Most common type of Venue,4th Most common type of Venue,5th Most common type of Venue
18,Central Toronto,0,Photography Studio,Park,Swim School,Bus Line,Diner


## Cluster 2

In [489]:
Toronto_merged.loc[Toronto_merged['Cluster labels']==1, Toronto_merged.columns[[1]+list (range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster labels,1th Most common type of Venue,2th Most common type of Venue,3th Most common type of Venue,4th Most common type of Venue,5th Most common type of Venue
0,Downtown Toronto,1,Coffee Shop,Pub,Bakery,Park,Café
1,Downtown Toronto,1,Coffee Shop,Clothing Store,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant
2,Downtown Toronto,1,Coffee Shop,Café,Cosmetics Shop,Cocktail Bar,Hotel
3,East Toronto,1,Asian Restaurant,Pub,Health Food Store,Trail,Discount Store
4,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Beer Bar
5,Downtown Toronto,1,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place
6,Downtown Toronto,1,Grocery Store,Café,Park,Baby Store,Coffee Shop
7,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Deli / Bodega,Gym
8,West Toronto,1,Bakery,Pharmacy,Café,Supermarket,Bank
10,Downtown Toronto,1,Coffee Shop,Aquarium,Café,Hotel,Fried Chicken Joint


## Cluster 3

In [491]:
Toronto_merged.loc[Toronto_merged['Cluster labels']==2, Toronto_merged.columns[[1]+list (range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster labels,1th Most common type of Venue,2th Most common type of Venue,3th Most common type of Venue,4th Most common type of Venue,5th Most common type of Venue
9,East York/East Toronto,2,Park,Coffee Shop,Convenience Store,Yoga Studio,Diner
33,Downtown Toronto,2,Park,Playground,Trail,Yoga Studio,Dessert Shop


## Cluster 4

In [492]:
Toronto_merged.loc[Toronto_merged['Cluster labels']==3, Toronto_merged.columns[[1]+list (range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster labels,1th Most common type of Venue,2th Most common type of Venue,3th Most common type of Venue,4th Most common type of Venue,5th Most common type of Venue
19,Central Toronto,3,Home Service,Fast Food Restaurant,Garden,Diner,Event Space


## Cluster 5

In [493]:
Toronto_merged.loc[Toronto_merged['Cluster labels']==4, Toronto_merged.columns[[1]+list (range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster labels,1th Most common type of Venue,2th Most common type of Venue,3th Most common type of Venue,4th Most common type of Venue,5th Most common type of Venue
29,Central Toronto,4,Playground,Tennis Court,Yoga Studio,Dessert Shop,Ethiopian Restaurant
